In [ ]:
!pip install -q accelerate -U
!pip install -q simpletransformers

In [ ]:
import os
import wandb
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from ast import literal_eval
import numpy as np
from dont_patronize_me import DontPatronizeMe
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import torch.functional as F
import tqdm
import data_preprocessing
import gensim
import transformers
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action
from nlpaug.util.file.download import DownloadUtil

# Disable wandb authorization request
os.environ["WANDB_START_METHOD"]="thread"
wandb.init(mode="disabled")

In [ ]:
random_seed = 42
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

device = 'cuda' if cuda_available else 'cpu'

print('Cuda available?', cuda_available)

PREPROCESSING_MODE = 'MEDIUM' # Choose between BASIC, MEDIUM and HEAVY
LOADING_MODE = 'k' # mode can be 'c', 'k', 'ck', ''

In [ ]:
def preprocess_data(data):
    data = data_preprocessing.remove_h_tags(data)
    data = data_preprocessing.remove_ampersands(data)
    data = data_preprocessing.remove_mentions(data)
    if PREPROCESSING_MODE != 'BASIC':
        data = data_preprocessing.lowercase(data)
    data = data_preprocessing.remove_contractions(data)
    if PREPROCESSING_MODE == 'HEAVY':
        data = data_preprocessing.remove_multiple_quotations(data)
    data = data_preprocessing.remove_extra_spaces(data)
    return data

def get_rows(dataframe, data):
    rows = [] # will contain par_id, label and text
    for idx in range(len(dataframe)):
        parid = dataframe.par_id[idx]
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        country = instance.country.values[0]
        text = instance.text.values[0]
        if LOADING_MODE == 'c':
            text = country + ' | ' + text
        elif LOADING_MODE == 'k':
            text = keyword + ' | ' + text
        elif LOADING_MODE == 'ck':
            text = country + ' | ' + keyword + ' | ' + text
        rows.append({
            'par_id':parid,
            'text':text,
            'label':instance.label.values[0]
        })
    return rows

def load_data(train_size=0.8, random_state=random_seed):
    dpm = DontPatronizeMe('./data', './data')
    dpm.load_task1()
    trids = pd.read_csv('data/train_semeval_parids-labels.csv')
    teids = pd.read_csv('data/dev_semeval_parids-labels.csv')
    trids.par_id = trids.par_id.astype(str)
    teids.par_id = teids.par_id.astype(str)
    data = dpm.train_task1_df
    data = preprocess_data(data)

    rows_train_val = get_rows(trids, data)
    rows_test = get_rows(teids, data)

    rows_train, rows_val = train_test_split(rows_train_val, train_size=train_size, random_state=random_state)
    print(len(rows_train), len(rows_val), len(rows_test))

    return pd.DataFrame(rows_train), pd.DataFrame(rows_val), pd.DataFrame(rows_test)

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        train_input_ids = self.encodings.input_ids[idx]
        train_token_type_ids = self.encodings.token_type_ids[idx]
        train_attention_mask = self.encodings.attention_mask[idx]
        train_labels = self.encodings.label[idx]
        return {
            'input_ids': train_input_ids,
            'token_type_ids': train_token_type_ids,
            'attention_mask': train_attention_mask,
            'labels': train_labels
        }

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
def data_augmentation(train_dataset_raw):
	# initialising the augmentor with "Glove"
	caug = naw.ContextualWordEmbsAug(
			# option to choose from is "word2vec", "glove" or "fasttext"
			model_path='distilroberta-base',

			# options available are insert or substitute
			action='substitute')
	train_dataset_raw_positive = train_dataset_raw[train_dataset_raw.label == 1]

	for idx, row in train_dataset_raw_positive.iterrows():
		train_dataset_raw.at[idx, 'text'] = caug.augment(row.text)

	train_dataset_raw = pd.concat([train_dataset_raw, train_dataset_raw_positive])
	#print(train_dataset_raw)
	train_dataset_raw.to_csv('data/augmented_train.csv', index=False)
	train_dataset_raw = pd.read_csv('data/augmented_train.csv')
	return train_dataset_raw
	

In [ ]:
train_dataset_raw, eval_dataset_raw, test_dataset_raw = load_data(train_size=0.8)

train_dataset_raw = data_augmentation(train_dataset_raw)

# Load the DeBERTa tokenizer
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

train_text = train_dataset_raw.text.values
eval_text = eval_dataset_raw.text.values
test_text = test_dataset_raw.text.values

encoding_train = tokenizer(train_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_eval = tokenizer(eval_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_test = tokenizer(test_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)

encoding_train['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in train_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_eval['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in eval_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_test['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in test_dataset_raw['label'].tolist()], dtype=torch.float32)

In [ ]:
# Create an instance of the CustomDataset class
train_dataset = CustomDataset(encoding_train)
eval_dataset = CustomDataset(encoding_eval)
test_dataset = CustomDataset(encoding_test)

In [ ]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
model = model.to(device)

epochs = 10
lr = 1e-5
max_length = 192
batch_size = 32

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(labels, axis=-1)
    f1 = f1_score(labels, predictions, average='binary')
    return {'f1': f1}

trainingargs = TrainingArguments(
    learning_rate=lr,
    weight_decay=1e-2,
    output_dir='/content/training_results',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    report_to=None,
    metric_for_best_model="f1",
    save_strategy='epoch',
    load_best_model_at_end=True,
    seed=random_seed,
    optim='adamw_torch',
)

trainer = Trainer(
    model=model,
    args=trainingargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# THESE CELLS HAVE BEEN RUN USING PREPROCESSING_MODE = 'MEDIUM'
assert PREPROCESSING_MODE == 'MEDIUM'

trainer.train()
# Here the best model has been already loaded
trainer.predict(test_dataset)